In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
#和整个训练相关的常量
#学习率
learningrate = 0.01
#epoch
epoch_size = 100
#batch_size
batch_size = 100
#tensorboard 日志目录
logs_path = "logs1"

#定义多层感知机模型里的常量
hidden_layer1 = 256
hidden_layer2 = 256


#数据特征
feature_size = 784
#类别数目
n_classes = 10

In [4]:
#输入占位符
def get_placeholder():
    train_x = tf.placeholder(dtype=tf.float32,shape=[None,784],name="InputData")
    train_y = tf.placeholder(dtype=tf.float32,shape=[None,10],name="LabelData")
    return train_x,train_y

In [5]:
def get_weights(shape):
    return tf.Variable(tf.truncated_normal(shape=shape,stddev=0.05),name="weights")

def get_bias(shape):
    return tf.Variable(tf.zeros(shape=shape),name = "bias")

In [6]:
def inference(train_x):
    with tf.name_scope("hidden1"):
        weights = get_weights(shape=[feature_size,hidden_layer1])
        bias = get_bias(shape=[hidden_layer1])
        layer1 = tf.nn.relu(tf.matmul(train_x,weights)+bias)
        tf.summary.histogram("layer1",layer1)
        
    with tf.name_scope("hidden2"):
        weights = get_weights(shape=[hidden_layer1,hidden_layer2])
        bias = get_bias(shape=[hidden_layer2])
        layer2 = tf.nn.relu(tf.matmul(layer1,weights)+bias)
        tf.summary.histogram("layer2",layer2)
    
    with tf.name_scope("output"):
        weights = get_weights(shape=[hidden_layer2,n_classes])
        bias = get_bias(shape=[n_classes])
        output = tf.add(tf.matmul(layer2,weights),bias)
    return output
    

In [7]:
def loss(output,train_y):
    with  tf.name_scope("LOSS"):
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=train_y))

In [8]:
def accuracy(output,train_y):
    acc = tf.equal(tf.arg_max(output,1),tf.arg_max(train_y,1))
    return tf.reduce_mean(tf.cast(acc,tf.float32))

#训练
train_x ,train_y = get_placeholder()

pred = inference(train_x)

cost = loss(pred,train_y)
acc = accuracy(pred,train_y)

with tf.name_scope("SGD"):
    optimizer = tf.train.GradientDescentOptimizer(learningrate)
    grads = tf.gradients(cost,tf.trainable_variables())
    grads = list(zip(grads,tf.trainable_variables()))
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)


#记录
tf.summary.scalar("loss",cost)
tf.summary.scalar("acc",acc)

for var in tf.trainable_variables():
    tf.summary.histogram(var.name,var)
    
merge_op = tf.summary.merge_all()

init= tf.global_variables_initializer()

W1115 10:31:11.757580 16564 deprecation.py:323] From <ipython-input-7-e17b38038a20>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W1115 10:31:12.013921 16564 deprecation.py:323] From <ipython-input-8-51995919eb44>:2: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


In [9]:
with tf.Session() as sess:
    print("training start...")
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logdir=logs_path,graph=tf.get_default_graph())
    batch_total = int(mnist.train.num_examples/batch_size)
    
    for epoch in range(epoch_size):
        
        avg_cost = 0
        
        for batch in range(batch_total):
            X,Y = mnist.train.next_batch(batch_size=batch_size)
            
            _,c,summary = sess.run([apply_grads,cost,merge_op],feed_dict={train_x:X,train_y:Y})
            
            avg_cost  += c/batch_total
            
            summary_writer.add_summary(summary,epoch*batch_total+batch_size)
            
        if (epoch+1)%5==0:
            print("epoch %d,cost = %f"%(epoch+1,avg_cost))
        
        

training start...
epoch 5,cost = 0.340684
epoch 10,cost = 0.257436
epoch 15,cost = 0.209423
epoch 20,cost = 0.175721
epoch 25,cost = 0.149715
epoch 30,cost = 0.129819
epoch 35,cost = 0.113661
epoch 40,cost = 0.100585
epoch 45,cost = 0.089468
epoch 50,cost = 0.080150
epoch 55,cost = 0.072292
epoch 60,cost = 0.065295
epoch 65,cost = 0.059209
epoch 70,cost = 0.053744
epoch 75,cost = 0.048900
epoch 80,cost = 0.044794
epoch 85,cost = 0.040907
epoch 90,cost = 0.037401
epoch 95,cost = 0.034292
epoch 100,cost = 0.031454
